In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
current_path = "/content/gdrive/MyDrive/APS360/APS360: Team Project/Project/Project Data/hatespeech"

#load ids from files
import glob
import pandas as pd 
files = glob.glob(current_path + "/p2p/*/*.csv")

ids = []
for file in files:
  loaded = pd.read_csv(file)
  ids.extend(loaded['tweet_id'].to_list())
print(ids) #29619 ids to get 
tweets = []

In [ ]:
import requests
import urllib.parse as urlparse
from urllib.parse import urlencode
import glob

endpoint = "https://api.twitter.com/2/tweets"


headers = {"Authorization": ""}
paths = []
paths = glob.glob("./p2p/twitter_hashtag_based_datasets/*.csv")

request = 0
while(request < min(300,ceil(len(ids)/100))) # 300 requests per 15 min window
  i = request*100
  upper_bound = min(i+100, len(ids))
  req_ids = ids[i:upper_bound] #get 100 tweets per req
  req_ep = add_params(endpoint, req_ids)
  r = requests.get(req_ep, headers=headers)
  try:
    response = r.json()
    print("successful")
    ogLength = len(tweets)
    for tweet in response['data']:
      tweets.append(tweet["text"])
    #print("added " + str(len(tweets) - ogLength)+ " to tweets. number of ids requested: " + str(len(req_ids)))
  except: # Unauthorized or deleted tweets
    print(r.text)



In [ ]:
import csv
with open(current_path + "/p2p/tweets.csv", 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(tweets) 

In [ ]:
def add_params(url, ids): # takes endpoint and list of ids
  ids_param = ','.join(map(str, ids))
  params = {"ids": ids_param}
  #print(params)
  url_parse = urlparse.urlparse(url)
  query = url_parse.query
  url_dict = dict(urlparse.parse_qsl(query))
  url_dict.update(params)
  url_new_query = urlparse.urlencode(url_dict)
  url_parse = url_parse._replace(query=url_new_query)
  new_url = urlparse.urlunparse(url_parse)

  #print(new_url)
  return new_url
